Colab notebook to train the BERT model for sentiment analysis in the vaccine data

# Install packages

In [1]:
# Install packages

%pip install torch
%pip install transformers
%pip install datasets
%pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

# Environment settings

In [2]:
# mount the colab session to google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Disabe W&B to conserve resources
import os
os.environ["WANDB_DISABLED"] = "true"

## Import packages

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from datasets import load_dataset
from datasets import load_metric

from wordcloud import WordCloud

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, DataCollatorWithPadding
from transformers import Trainer

# Prepare dataset



In [5]:
train_df_path =  "/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/data/train_subset.csv"
val_df_path = "/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/data/eval_subset.csv"

train_raw_df = pd.read_csv(train_df_path)
val_raw_df = pd.read_csv(val_df_path)

In [6]:
train_raw_df.head()

,tweet_id,safe_text,label,agreement,tweet_length
0,YMRMEDME,Mickey's Measles has gone international <url>,Neutral,1.000000,45
1,5GV8NEZS,S1256 [NEW] Extends exemption from charitable ...,Neutral,1.000000,122
2,EI10PS46,<user> your ignorance on vaccines isn't just ...,Positive,0.666667,118
3,OM26E6DG,Pakistan partly suspends polio vaccination pro...,Neutral,1.000000,93
4,NBBY86FX,In other news I've gone up like 1000 mmr,Neutral,1.000000,40


In [7]:
# drop the null rows, if any

train_df = train_raw_df.dropna()
val_df = val_raw_df.dropna()

In [8]:
print("Train data shape : ", train_df.shape)
print("Val data shape", val_df.shape)

Train data shape :  (7999, 5)
Val data shape (2000, 5)


In [9]:
train_df.head()

,tweet_id,safe_text,label,agreement,tweet_length
0,YMRMEDME,Mickey's Measles has gone international <url>,Neutral,1.000000,45
1,5GV8NEZS,S1256 [NEW] Extends exemption from charitable ...,Neutral,1.000000,122
2,EI10PS46,<user> your ignorance on vaccines isn't just ...,Positive,0.666667,118
3,OM26E6DG,Pakistan partly suspends polio vaccination pro...,Neutral,1.000000,93
4,NBBY86FX,In other news I've gone up like 1000 mmr,Neutral,1.000000,40


In [10]:
val_df.head()

,tweet_id,safe_text,label,agreement,tweet_length
0,R7JPIFN7,Children's Museum of Houston to Offer Free Vac...,Positive,1.000000,61
1,2DD250VN,<user> no. I was properly immunized prior to t...,Positive,1.000000,102
2,ESEVBTFN,<user> thx for posting vaccinations are impera...,Positive,1.000000,120
3,S17ZU0LC,This Baby Is Exactly Why Everyone Needs To Vac...,Positive,0.666667,69
4,IIN5D33V,"Meeting tonight, 8:30pm in room 322 of the stu...",Positive,1.000000,118


In [11]:
# save the data

train_path = "/content/train.csv"
val_path = "/content/val.csv"

train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)

In [12]:
# Load the dataset in format for BERT training

dataset = load_dataset('csv',
                        data_files={'train': train_path,
                                    'val': val_path},
                       encoding = "ISO-8859-1")


Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

**BERT tokenizer**

In [13]:
# Create a BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
# Function to map labels for BERT training
def map_labels(data):
  label = data['label']

  encoding = 0
  if label == 'Negative': # -1
    encoding = 0
  elif label == 'Neutral': # 0
    encoding = 1
  elif label == 'Positive': # 1
    encoding = 2

  return {'labels': encoding}


# Function to tokenize tweets
def tokenize_data(example):
  return tokenizer(example['safe_text'], padding='max_length', truncation=True, max_length=256)


# Do the tokenization of dataset
dataset = dataset.map(tokenize_data, batched=True)


# Do the final transformation
removable_columns = ['tweet_id', 'safe_text', 'label', 'agreement']
dataset = dataset.map(map_labels, remove_columns=removable_columns)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [15]:
# View the dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['tweet_length', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7999
    })
    val: Dataset({
        features: ['tweet_length', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [16]:
# split the main dataset with train and val set

train_dataset = dataset['train'].shuffle(seed=5)
val_dataset = dataset['val'].shuffle(seed=0)

In [17]:
# create pytorch tensors with padding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training configuration and model creation

In [24]:
weight_fol = "/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/BERT/final/weights"
log_fol = "/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/BERT/final/logs"

logging_steps = 500
epochs = 2

In [25]:
# Configure training parameters and hyper-parameters

training_args = TrainingArguments(
    weight_fol,  # folder to save weights
    logging_dir=log_fol, # Directory for logs
    logging_steps=logging_steps, # logging interval

    num_train_epochs=epochs,  # no of epochs

    load_best_model_at_end=True,  # load best model at training end
    evaluation_strategy="steps",
    save_strategy="steps",
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [26]:
# Define metric

metric = load_metric("accuracy")

def compute_metrics(eval_predictions):
  logits, labels = eval_predictions
  predictions = np.argmax(logits, axis=-1)

  return metric.compute(predictions=predictions, references=labels)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


**BERT model**

In [27]:
# Load pretrained BERT model to fine tune

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# create trainer

trainer = Trainer(
  model = model,
  tokenizer = tokenizer,
  train_dataset = train_dataset,
  eval_dataset = val_dataset,
  data_collator = data_collator,
  compute_metrics = compute_metrics,
  args = training_args,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


# Train

In [ ]:
# Start the training

trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.774800,0.681420,0.724500


Step,Training Loss,Validation Loss,Accuracy
500,0.774800,0.681420,0.724500
1000,0.677400,0.667948,0.745000
1500,0.591300,0.632129,0.754000


# Evaluate

In [30]:
# Evaluate model

trainer.evaluate()

{'eval_loss': 0.6228652000427246,
 'eval_accuracy': 0.763,
 'eval_runtime': 35.4814,
 'eval_samples_per_second': 56.368,
 'eval_steps_per_second': 7.046,
 'epoch': 2.0}

Evaluation accuracy : **76.3%**

# Save final weights

In [31]:
# Save the trained model and tokenizer

model.save_pretrained("/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/BERT/final/final_weights/model")
tokenizer.save_pretrained("/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/BERT/final/final_weights/tokenizer")

('/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/BERT/final/final_weights/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/BERT/final/final_weights/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/BERT/final/final_weights/tokenizer/vocab.txt',
 '/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/BERT/final/final_weights/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/University/NLP/Sentiment_Analysis__vaccine_data/BERT/final/final_weights/tokenizer/tokenizer.json')